Do some calculations for the windowed model.

In [46]:
# where are individual predictions stored?
predictions_folder = '/Users/jnaiman/Downloads/tmp/ocrpost/eval/predicted/'

# which set of test instances to use
test_file_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/alignments/'
#test_file = 'test_file_small_words_pageLevel.csv'
test_file = 'test_file_small_words_pageLevel_old.csv'

In [65]:
from glob import glob
import pandas as pd
import fastwer
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib

In [69]:
def get_iqr(df_byt5_words,col):
    colvalue = df_byt5_words[col]
    colvalue = colvalue[~np.isnan(colvalue)]
    Q3 = np.quantile(colvalue, 0.75)
    Q1 = np.quantile(colvalue, 0.25)
    IQR = Q3 - Q1
    return IQR

def get_iqr_by_value(colvalue):
    Q3 = np.quantile(colvalue, 0.75)
    Q1 = np.quantile(colvalue, 0.25)
    IQR = Q3 - Q1
    return IQR

def get_stats(df_byt5_words,col, round_val = 2, print_nans=False):
    colvalue = df_byt5_words[col]
    colvalue = colvalue[~np.isnan(colvalue)]
    # how many nan's
    if print_nans: print('# NaNs =', len(df_byt5_words[col])-len(colvalue))
    med = colvalue.median()
    mea = colvalue.mean()
    std = colvalue.std()
    iqr = get_iqr_by_value(colvalue)
    return round(mea,round_val), round(med,round_val), round(std,round_val), round(iqr,round_val)

def get_improvement_stats(df_byt5_words,col_before, col_after, round_val = 2):
    colvalueb = df_byt5_words[col_before]
    colvalueb = colvalueb[~np.isnan(colvalueb)]
    colvaluea = df_byt5_words[col_after]
    colvaluea = colvaluea[~np.isnan(colvaluea)]
    mask = colvalueb != 0
    colvalue = 100*(1-colvaluea[mask]/colvalueb[mask])
    med = colvalue.median()
    mea = colvalue.mean()
    std = colvalue.std()
    iqr = get_iqr_by_value(colvalue)
    return round(mea,round_val), round(med,round_val), round(std,round_val), round(iqr,round_val) 

In [49]:
test_data = pd.read_csv(test_file_dir+test_file)

In [50]:
test_data.head()

,gt,ocr,filename,page,gt_clean,ocr_clean,ocr_align,pdf_align,pdf_align_type
0,across a flare. The light curves in Fig. 6 cle...,across a f@are. The light curves in Fie. 6 cle...,0401_astro-ph0401222d_preprin_psfilefixedRTM_hocr,page6,across a flare. The light curves in Fig. 6 cle...,across a fare. The light curves in Fie. 6 clea...,across a f^are. The light curves in Fie. 6 cle...,across a flare. The light curves in Fig. 6 cle...,WWWWWW W WWWWWW WWW WWWWW WWWWWW WW WWWW W WWW...
1,We also verified that the non-gray corrections...,We also verified that the non-gray corrections...,1106_1106.6007d_ms_psfilefixedRTM_hocr,page5,We also verified that the non-gray corrections...,We also verified that the non-gray corrections...,We also verified that the non-gray ^^log^^y co...,We also verified that the non-gray $\log g$ co...,WW WWWW WWWWWWWW WWWW WWW WWWWWWWW IIIIIIII WW...
2,Figure shows that the lag increases with the l...,Figure shows that the lag increases with the l...,0505_astro-ph0505276d_mnsubm_psfilefixedRTM_hocr,page6,Figure shows that the lag increases with the l...,Figure shows that the lag increases with the l...,Figure ^^^^^^^^^^^^^^5 shows that the lag incr...,Figure \ref{lag_alpha} shows that the lag incr...,WWWWWW RRRRRRRRRRRRRRR WWWWW WWWW WWW WWW WWWW...
3,@@@@@.@@ Accurate distances...,2001)... Accurate distances...,0910_0910.5253d_millerjones_v404_paralla_psfil...,page0,. Accurate distances are al...,2001)... Accurate distances...,(Revuoldsetal.2008).. Jouker&Nelemans(2001). ...,"\cit@@@@@ep{Jon04}@.@ \citep{Rey08}, \citet{J...",cccccccccccccccccccWW CCCCCCCCCCCCCW CCCCCCCC...
4,is on a length scale sm@aller than about one f...,is on a leneth scale siualler than about one f...,0408_astro-ph0408115d_haverkorn_aug04_psfilefi...,page10,is on a length scale smaller than about one fi...,is on a leneth scale siualler than about one f...,is on a leneth scale siualler than about one f...,is on a length scale sm@aller than about one f...,WW WW W WWWWWW WWWWW WWWWWWWW WWWW WWWWW WWW W...


In [51]:
cer_orig = []; wer_orig = []
for i in range(len(test_data)):
    d = test_data.iloc[i]
    ocr = d['ocr_clean']
    pdf = d['gt_clean']
    if pd.isnull(ocr) or pd.isnull(pdf):
        cer_orig.append(np.nan)
        wer_orig.append(np.nan)
        continue
        
    ocr = str(ocr); pdf = str(pdf)
    cer = fastwer.score_sent(ocr,pdf,char_level=True)
    wer = fastwer.score_sent(ocr,pdf,char_level=False)
    
    cer_orig.append(cer)
    wer_orig.append(wer)  
test_data['CER Orig'] = cer_orig
test_data['WER Orig'] = wer_orig

In [81]:
pfiles = glob(predictions_folder + '*.pickle')
pfiles

['/Users/jnaiman/Downloads/tmp/ocrpost/eval/predicted/distjoint_greedy2.pickle',
 '/Users/jnaiman/Downloads/tmp/ocrpost/eval/predicted/distjoint_greedy1.pickle']

In [ ]:
# get all dealies
for pf in pfiles:
    pfn = pf.split('/')[-1].split('.pickle')[0]
    with open(pf,'rb') as f:
        corrections,raw = pickle.load(f)
    if len(corrections) != len(test_data):
        print('not correct number of outputs!')
        import sys; sys.exit()
                
    print('on', pfn)
    cer_orig = []; wer_orig = []
    for i in range(len(test_data)):
        d = test_data.iloc[i]
        #ocr = d['ocr_clean']
        ocr = corrections[i].replace('@','') # 'fixed' ocr
        pdf = d['gt_clean']
        if pd.isnull(ocr) or pd.isnull(pdf):
            cer_orig.append(np.nan)
            wer_orig.append(np.nan)
            continue

        ocr = str(ocr); pdf = str(pdf)
        cer = fastwer.score_sent(ocr,pdf,char_level=True)
        wer = fastwer.score_sent(ocr,pdf,char_level=False)
        cer_orig.append(cer)
        wer_orig.append(wer)
    test_data['CER ' + pfn] = cer_orig
    test_data['WER ' + pfn] = cer_orig
print('all done!')

on distjoint_greedy2


In [ ]:
fig, ax = plt.subplots(1,len(pfiles),figsize=(5,5*len(pfiles)))
if type(ax) == matplotlib.axes._axes.Axes:
    ax = [ax]

for ip,pf in enumerate(pfiles):
    pfn = pf.split('/')[-1].split('.pickle')[0]
    ax[ip].plot(test_data['CER Orig'], test_data['CER ' + pfn], 'o')
    ax[ip].plot([test_data['CER Orig'].min(),test_data['CER Orig'].max()],[test_data['CER Orig'].min(),test_data['CER Orig'].max()], color='orange')

In [ ]:
for ip,pf in enumerate(pfiles):
    pfn = pf.split('/')[-1].split('.pickle')[0]
    before = 'Orig'
    after = pfn
    print('********************', pfn, '*********************')

    print('--------- Before -------------')
    mea_cbefore,med_cbefore,std_cbefore,iqr_cbefore = get_stats(test_data,'CER '+before, print_nans=True)
    print('CER : mean =', mea_cbefore, ', std =', std_cbefore, ', median =',med_cbefore, ', iqr =', iqr_cbefore)
    mea_wbefore,med_wbefore,std_wbefore,iqr_wbefore = get_stats(test_data,'WER '+before)
    print('WER : mean =', mea_wbefore, ', std =', std_wbefore, ', median =',med_wbefore, ', iqr =', iqr_wbefore)
    print('')
    print('--------- After -------------')
    mea_cafter,med_cafter,std_cafter,iqr_cafter = get_stats(test_data,'CER '+after)
    print('CER : mean =', mea_cafter, ', std =', std_cafter, ', median =',med_cafter, ', iqr =', iqr_cafter)
    mea_wafter,med_wafter,std_wafter,iqr_wafter = get_stats(test_data,'WER '+after)
    print('WER : mean =', mea_wafter, ', std =', std_wafter, ', median =',med_wafter, ', iqr =', iqr_wafter)
    print('')
    # improvement
    #improvement = lambda df: 100 * (1 - df.cer_after / df.cer_before)
    mea_c,med_c,std_c,iqr_c = get_improvement_stats(test_data,'CER '+before, 'CER '+after)
    print('------------ Improvement ----------------')
    print('CER : mean = ', mea_c, ', std =', std_c, ', median =',med_c, ', iqr =', iqr_c)
    mea_w,med_w,std_w,iqr_w = get_improvement_stats(test_data,'WER '+before, 'WER '+after)
    print('WER : mean = ', mea_w, ', std =', std_w, ', median =',med_w, ', iqr =', iqr_w)
    print('')
    print('')